## インポート

In [1]:

import sys
import re
import difflib
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

from config import (NUMBERS,
                                   NUMBERS3,
                                   NUMBERS4,
                                   NUMBERS4_RESULT,
                                   NUMBERS4_VECTORS,
                                   NUMBERS4_PATTERNS)
import utils


%load_ext autoreload
%autoreload 2

#### NUMBERS4 で予測する

In [2]:
X = []
Y = []
key_0 = 0
key_1 = 0
key_2 = 0
key_3 = 0
key_4 = 0
key_5 = 0

key_6  = 0 #  赤口
key_7  = 0 #  仏滅
key_8  = 0 #  友引
key_9  = 0 #  先勝
key_10 = 0 #  先負
key_11 = 0 #  大安

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4[_i][_time]
    _z = NUMBERS[_i][4]
    for _k, _v in enumerate(_x):
        # if len(list(set(_y) & set(_v))) >= 3:
        if len(list(set(_y) & set(_v))) >= 3 and _z == "先負":
            if _k == 0:
                key_0 += 1
            elif _k == 1:
                key_1 += 1
            elif _k == 2:
                key_2 += 1
            elif _k == 3:
                key_3 += 1
            elif _k == 4:
                key_4 += 1
            elif _k == 5:
                key_5 += 1
                
            if _z == "赤口":
                key_6 += 1
            elif _z == "仏滅":
                key_7 += 1
            elif _z == "友引":
                key_8 += 1
            elif _z == "先勝":
                key_9 += 1
            elif _z == "先負":
                key_10 += 1
            elif _z == "大安":
                key_11 += 1
            
            
            _format = "{0}回 {1}:{2}({3}) [{4}]".format(_time, _y, _x[_k], _k, _z)
            # print(_format)
    # Y.append(_y)
    # X.append(_x)
    pass
_format1 = "前回:{0}個, 前々回:{1}個, 前回曜日:{2}個, 前々回曜日:{3}個, 前回六曜:{4}個, 前々回六曜:{5}個".format(key_0, key_1, key_2, key_3, key_4, key_5)
_format2 = "赤口:{0}個, 仏滅:{1}個, 友引:{2}個 先勝:{3}個, 先負:{4}個, 大安:{5}個".format(key_6, key_7, key_8, key_9, key_10, key_11)
_sum = sum([key_0, key_1, key_2, key_3, key_4, key_5])

print(_format1)

前回:9個, 前々回:7個, 前回曜日:8個, 前々回曜日:5個, 前回六曜:8個, 前々回六曜:14個


In [9]:
X = []
Y = []
Z = []
LOCK = {}
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS4[_i][_time]
    lock = []
    hook = []
    key_0 = 0
    key_1 = 0
    key_2 = 0
    key_3 = 0
    key_4 = 0
    key_5 = 0
    key_6 = 0
    key_7 = 0
    key_8 = 0
    key_9 = 0
    for _n in _x:
        if _time == _time:
            _h = list(_n)
            for _j in _h:
                if _j == '0':
                    key_0 += 1
                elif _j == '1':
                    key_1 += 1
                elif _j == '2':
                    key_2 += 1
                elif _j == '3':
                    key_3 += 1
                elif _j == '4':
                    key_4 += 1
                elif _j == '5':
                    key_5 += 1
                elif _j == '6':
                    key_6 += 1
                elif _j == '7':
                    key_7 += 1
                elif _j == '8':
                    key_8 += 1
                elif _j == '9':
                    key_9 += 1
            lock.append(list(_n))
    data = {
        "0": str(int(key_0) + 1),
        "1": str(int(key_1) + 1),
        "2": str(int(key_2) + 1),
        "3": str(int(key_3) + 1),
        "4": str(int(key_4) + 1),
        "5": str(int(key_5) + 1),
        "6": str(int(key_6) + 1),
        "7": str(int(key_7) + 1),
        "8": str(int(key_8) + 1),
        "9": str(int(key_9) + 1),
    }
    LOCK.update({ _time: [key_0, key_1, key_2, key_3, key_4, key_5, key_6, key_7, key_8, key_9, _y]})
    Z.append(data)
    X.append(lock)
    
lock = pd.DataFrame(LOCK)

In [10]:
LOCK

{'4960': [1, 1, 3, 3, 3, 3, 2, 4, 2, 2, '625'],
 '4959': [4, 1, 2, 1, 1, 1, 2, 4, 6, 2, '061'],
 '4958': [0, 2, 0, 4, 4, 1, 1, 3, 4, 5, '870'],
 '4957': [1, 0, 0, 3, 2, 2, 6, 3, 4, 3, '668'],
 '4956': [4, 1, 1, 4, 1, 1, 3, 3, 3, 3, '901'],
 '4955': [2, 3, 2, 1, 2, 3, 3, 3, 4, 1, '489'],
 '4954': [2, 3, 3, 4, 2, 2, 1, 3, 1, 3, '193'],
 '4953': [1, 3, 1, 3, 2, 2, 3, 6, 1, 2, '349'],
 '4952': [2, 4, 2, 2, 3, 3, 2, 2, 2, 2, '751'],
 '4951': [2, 3, 3, 2, 2, 1, 1, 4, 1, 5, '032'],
 '4950': [2, 3, 4, 2, 1, 2, 3, 4, 0, 3, '256'],
 '4949': [1, 3, 1, 4, 3, 2, 3, 3, 1, 3, '549'],
 '4948': [4, 2, 1, 3, 1, 2, 2, 3, 2, 4, '001'],
 '4947': [2, 7, 4, 2, 1, 2, 1, 3, 1, 1, '018'],
 '4946': [1, 5, 3, 3, 2, 2, 1, 3, 0, 4, '912'],
 '4945': [2, 4, 0, 4, 1, 3, 4, 3, 1, 2, '182'],
 '4944': [2, 6, 2, 3, 1, 2, 1, 4, 1, 2, '119'],
 '4943': [2, 5, 1, 4, 2, 3, 2, 1, 0, 4, '765'],
 '4942': [2, 3, 2, 2, 1, 3, 4, 2, 1, 4, '487'],
 '4941': [3, 1, 3, 6, 1, 2, 3, 4, 1, 0, '642'],
 '4940': [3, 2, 1, 3, 3, 2, 4, 2, 2, 2, 

#### 6. 抽選数を特徴量とする

In [11]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS4_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x = NUMBERS4_RESULT[_i][_time]
    _z = NUMBERS[_i][4]
    if _z == "先勝":
        Y.append(_y)
        X.append(_x)
        TIME.append(_time)

In [ ]:
from statistics import mean, median,variance,stdev

X0 = []
X1 = []
X2 = []
X3 = []
for _i in range(len(NUMBERS4_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x0 = NUMBERS4_RESULT[_i][_time][0]
    _x1 = NUMBERS4_RESULT[_i][_time][1]
    _x2 = NUMBERS4_RESULT[_i][_time][2]
    _x3 = NUMBERS4_RESULT[_i][_time][3]
    X0.append(int(_x0))
    X1.append(int(_x1))
    X2.append(int(_x2))
    X3.append(int(_x3))
    

x0 = int(mean(X0))
x1 = int(mean(X1))
x2 = int(mean(X2))
x3 = int(mean(X3))
print(x0, x1, x2, x3)

# 20180618
# data = [float("1"), float("1"), float("1"), float("1")]
data = [float(str(x0)), float(str(x1)), float(str(x2)), float(str(x3))]

#### 設定

In [213]:
OLD_WEEK = "友引"
TYPE            = "-"

In [347]:
PATTERN = "pattern{0}".format(9)

##### 特徴量

In [348]:
VEC = utils.get_vectors(OLD_WEEK, TYPE, NUMBERS4_VECTORS)
NUM  = utils.get_patterns(NUMBERS4_PATTERNS, PATTERN, TYPE)
print(NUM, VEC)

[1, 2] [0, 4, 5]


##### セット

In [349]:
X       = []
Y       = []
TIME = []
k1 = VEC[0]
k2 = VEC[1]
k3 = VEC[2]
n1 = NUM[0]
n2 = NUM[1]

##### 計算

In [350]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][3]
    _x1 = NUMBERS4[_i][_time][k1]
    _x2 = NUMBERS4[_i][_time][k2]
    _x３ = NUMBERS4[_i][_time][k3]
    _z = NUMBERS[_i][4]
    if not len(list(set(_y) & set(_x1))) >= n1 and not len(list(set(_y) & set(_x2))) >= n2  and _z == OLD_WEEK:
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)

## 予測

In [351]:
y = pd.DataFrame(Y)
x = pd.DataFrame(X)

In [352]:
len(y)

29

In [353]:
len(TIME)

29

In [354]:
model = linear_model.LinearRegression()
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [355]:
px = x
py = model.predict(px)

In [356]:
for _i in range(len(py)):
    _py = str(int(py[_i]))
    for _k, _v in enumerate(NUMBERS):
        if _v[0] == TIME[_i]:
            if len(list(set(_v[3]) & set(_py))) >= 3:
                print(TIME[_i], _v[3], py[_i])
            # print(TIME[_i], _v[3], py[_i])

## 決定係数

In [357]:
r2_score(y, py)

0.056764995882303104

In [358]:
model.score(x, y)

0.056764995882303104

In [359]:
model.coef_[0]

-0.140519953560098

## 予測

In [360]:
pkl_file = "data/numbers4_20180711.pkl"

In [361]:
data = [float("3155"), float("8943"), float("7835")]

In [362]:
joblib.dump(model, pkl_file)

['data/numbers4_20180711.pkl']

In [363]:
clf = joblib.load(pkl_file)

In [364]:
processor = clf.predict([data])

In [365]:
processor

array([5497.07920269])